# Web Scraping Weather Data
Webscraping tutorial: https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [1]:
import requests
import lxml.html as lh
import pandas as pd

## 1. Define Web Scraping Function

In [2]:
url = 'https://www.hko.gov.hk/en/wxinfo/pastwx/ywx2014.htm'

# create a handle, page, to handle the contents
page = requests.get(url)

# store contents of website under doc
doc = lh.fromstring(page.content)

# parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
# Check length of list
print('Length of list:', len(tr_elements))

# Notice the list is 49, so it has captured all three tables
# We only need elements from table 1
# Determine which row has two items in the list -- this is the row that lists the station
res = [] 
for idx in range(0, len(tr_elements)) : 
    if len(tr_elements[idx]) == 2: 
        res.append(idx)
station_idx = res[0]
print('Index of station name:', station_idx)

Length of list: 49
Index of station name: 16


In [4]:
# Get tr elements of only first table
print([len(T) for T in tr_elements[:station_idx+1]])

[7, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 2]


**Get Header for 1st Table**

In [5]:
# create empty list
col=[]

# for each row, store each first element (header) and an empty list
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
    
j = 0
delete = col[2]

for t in tr_elements[1]:
    name=t.text_content()
    col.insert(j+2,(name + ' Air Temperature',[]))
    j+=1

# Remove column with Temperature Title
col.remove(delete)

for column in col:
    print(col.index(column),column)

0 ('Month', [])
1 ('MeanPressure(hPa)', [])
2 ('Mean DailyMax.(deg. C) Air Temperature', [])
3 ('Mean(deg. C) Air Temperature', [])
4 ('Mean DailyMin.(deg. C) Air Temperature', [])
5 ('MeanDew Point(deg. C)', [])
6 ('MeanRelativeHumidity(%)', [])
7 ('MeanAmountof Cloud(%)', [])
8 ('TotalRainfall(mm)', [])


In [6]:
# Collect data from each row
# Since header data is stored in 0-1, we start at index 2
for j in range(2,len(tr_elements)):
    # T is the j'th row
    T = tr_elements[j]
    
    # If row is not of size 9, the //tr data is not from table 1
    if len(T)!=9:
        break
        
    # i is the index of our column
    i=0
    
    # Iterate througgh each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        # Check if row is empty
        if i>0:
            # Convert any numerical value to integer
            try:
                data=int(data)
            except:
                pass
        # Append the data to the empty list of the i'th column
        col[i][1].append(data)
        # Increment i for the next column
        i+=1

In [7]:
# Check length of each column
# They should all be the same
[len(C) for (title,C) in col]

[14, 14, 14, 14, 14, 14, 14, 14, 14]

In [8]:
# Create DataFrame of weather data
Dict = {title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df['Year'] = 2014
df.tail(3)

,Month,MeanPressure(hPa),Mean DailyMax.(deg. C) Air Temperature,Mean(deg. C) Air Temperature,Mean DailyMin.(deg. C) Air Temperature,MeanDew Point(deg. C),MeanRelativeHumidity(%),MeanAmountof Cloud(%),TotalRainfall(mm),Year
11,Dec,1021.7,18.5,16.3,14.2,9.9,67,67,44.7,2014
12,Mean/Total,1013.1,26.0,23.5,21.5,19.3,78,66,2638.3,2014
13,Normal*,1012.9,25.6,23.3,21.4,19.0,78,68,2398.5,2014


In [9]:
# Remove last two rows
df = df.iloc[:-2,:]
df.tail(3)

,Month,MeanPressure(hPa),Mean DailyMax.(deg. C) Air Temperature,Mean(deg. C) Air Temperature,Mean DailyMin.(deg. C) Air Temperature,MeanDew Point(deg. C),MeanRelativeHumidity(%),MeanAmountof Cloud(%),TotalRainfall(mm),Year
9,Oct,1014.6,28.9,26.2,24.3,20.4,71,54,109.8,2014
10,Nov,1017.0,24.6,22.6,21.2,18.5,78,63,31.1,2014
11,Dec,1021.7,18.5,16.3,14.2,9.9,67,67,44.7,2014


In [10]:
# Create month dictionary to convert to numbers
d={'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}

# Create Year_Month column for analysis
df['Year_Month'] = df['Year'].astype(str).str.cat(df['Month'].map(d).astype(str),sep="-")
df.head(3)

,Month,MeanPressure(hPa),Mean DailyMax.(deg. C) Air Temperature,Mean(deg. C) Air Temperature,Mean DailyMin.(deg. C) Air Temperature,MeanDew Point(deg. C),MeanRelativeHumidity(%),MeanAmountof Cloud(%),TotalRainfall(mm),Year,Year_Month
0,Jan,1021.3,19.2,16.3,14.1,9.9,67,32,Trace,2014,2014-1
1,Feb,1017.7,17.9,15.5,13.5,12.3,82,73,39.5,2014,2014-2
2,Mar,1017.1,20.9,18.7,17.0,15.7,83,77,207.6,2014,2014-3


In [11]:
# Reorder df content
df = df[['Year_Month','MeanPressure(hPa)', 'Mean DailyMax.(deg. C) Air Temperature',
       'Mean(deg. C) Air Temperature',
       'Mean DailyMin.(deg. C) Air Temperature', 'MeanDew Point(deg. C)',
       'MeanRelativeHumidity(%)', 'MeanAmountof Cloud(%)', 'TotalRainfall(mm)']]
df.head(3)

,Year_Month,MeanPressure(hPa),Mean DailyMax.(deg. C) Air Temperature,Mean(deg. C) Air Temperature,Mean DailyMin.(deg. C) Air Temperature,MeanDew Point(deg. C),MeanRelativeHumidity(%),MeanAmountof Cloud(%),TotalRainfall(mm)
0,2014-1,1021.3,19.2,16.3,14.1,9.9,67,32,Trace
1,2014-2,1017.7,17.9,15.5,13.5,12.3,82,73,39.5
2,2014-3,1017.1,20.9,18.7,17.0,15.7,83,77,207.6


In [27]:
import os
path = os.getcwd()
parent = os.path.dirname(path) 
print(parent)

df.to_csv(parent+'/data/raw/hk_weather_2014.csv')

/Users/tiffanyflor/Dropbox/MyProjects/HongKongPollution/HongKongPollution
